In [80]:
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:
! pip install --upgrade jupyter ipywidgets

In [81]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from ragchecker.integrations.llama_index import response_to_rag_results
from ragchecker import RAGResults, RAGChecker
from ragchecker.metrics import all_metrics

In [ ]:
! pip install llama-index-readers-web


<hr>

Scrap the web page for answer the certain question

In [82]:

from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display

In [83]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://www.ml.school/"]
)

In [84]:
documents[0]

Document(id_='http://www.ml.school/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4 \\- 21, 2024\n\nCheck the schedule for more details about upcoming cohorts.\n\nI want to join!Sign in\n\n## Learn how to **design** , **build** , **deploy** , and **scale** machine\nlearning systems to solve real-world problems.\n\nI\'ll lose my mind if I see another book or course teaching people the same\nbasic ideas for the hundredth time. Most people are stuck in beginner mode,\nand finding help to solve real-world problems is hard.\n\nI want to change that.\n\nI started writing software 30 years ago. I\'ve 

In [85]:
index = SummaryIndex.from_documents(documents)

In [86]:
# Create query engine
rag_application = index.as_query_engine()

In [61]:
response1 = rag_application.query("Who teaches this course?")

print(response1)

Santiago teaches this course.


In [62]:
response1

Response(response='Santiago teaches this course.', source_nodes=[NodeWithScore(node=TextNode(id_='f18d15ae-97e1-4900-a2bc-0a1388ab09c4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='http://www.ml.school/', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='01d5d29cc37a267e7fd48034058b35d6aba880a539ad3e294c217e45b98a0413'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='454c4df6-61f5-4975-9161-a48e75584616', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='3442213380f08644e4288760e626113c4232b05faf77697904e43d5108da3ccf')}, text='> "This is the best machine learning course I\'ve done. Worth every cent."\n\n**Jose Reyes** , AI/ML at Cevo Australia\n\n# Building Machine Learning Systems That Don\'t Suck\n\nA **live, interactive program** that\'ll help you build **production-ready**\nmachine learning systems from the ground up.\n\nNext cohort: November 4

the query engine is working
<hr>

Using the original method to get the rag_results for evaluation

- But it failed
- successfully got the response and transform it to `rag_result`
- but failed to create `rag_results`

In [87]:
# User query and groud truth answer
user_query = "What is Price of this course?"
gt_answer = "The price of the course is $500"


# Get response from LlamaIndex
response_object = rag_application.query(user_query)

# Convert to RAGChecker format
rag_result = response_to_rag_results(
    query=user_query,
    gt_answer=gt_answer,
    response_object=response_object,
)

# Create RAGResults object
rag_results = RAGResults.from_dict({"results": [rag_result]})
print(rag_results)

RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)


/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/eval/venv/lib/python3.11/site-packages/dataclasses_json/core.py:201: RuntimeWarning: 'NoneType' object value of non-optional type query_id detected when decoding RAGResult.
  warnings.warn(


Create a json file for the rag_results
- But it failed
- Generate the json file for the `rag_result`


In [77]:
with open("/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/eval/rag_response.json") as fp:
    rag_results = RAGResults.from_json(fp.read())


In [78]:
print(rag_results)

RAGResults(
  2 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)


No matter what method I use, it failed

In [90]:


# Note the prefix "openai/" here
model = "gpt-4o"
extractor = model
checker = model

evaluator = RAGChecker(
    # extractor_name="bedrock/meta.llama3-70b-instruct-v1:0",
    # checker_name="bedrock/meta.llama3-70b-instruct-v1:0",
    extractor_name=extractor,
    checker_name=checker,
)



In [91]:
# Evaluate using RAGChecker
evaluator.evaluate(rag_results, all_metrics)


2024-10-08 22:54:50.949 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for gt_answer of 1 RAG results.
  0%|          | 0/1 [00:00<?, ?it/s]

litellm.BadRequestError: OpenAIException - Error code: 400 - {'error': {'message': 'Unrecognized request arguments supplied: api_url, max_workers', 'type': 'invalid_request_error', 'param': None, 'code': None}} [sleep 10 seconds]


  0%|          | 0/1 [00:04<?, ?it/s]


KeyboardInterrupt: 

Clear to see that the evaluation is not working

In [37]:
evaluator = RAGChecker(
    extractor_name="bedrock/meta.llama3-70b-instruct-v1:0",
    checker_name="bedrock/meta.llama3-70b-instruct-v1:0",
    batch_size_extractor=32,
    batch_size_checker=32,
)

# Check if rag_results contains valid data
if rag_results.results and all(result.response for result in rag_results.results):
    try:
        evaluator.evaluate(rag_results, all_metrics)
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
else:
    print("rag_results does not contain valid data for evaluation")

# Print detailed results
print(rag_results)

2024-10-08 16:17:25.714 | INFO     | ragchecker.evaluator:extract_claims:113 - Extracting claims for gt_answer of 1 RAG results.
  0%|          | 0/1 [00:02<?, ?it/s]

Exit with the following error:
'NoCredentialsError' object has no attribute 'response'
An error occurred during evaluation: 'NoneType' object is not iterable
RAGResults(
  1 RAG results,
  Metrics:
  {
    "overall_metrics": {},
    "retriever_metrics": {},
    "generator_metrics": {}
  }
)
